###HW14.3 Field-aware Factorization Machine  FFM test
**Download the Spark libFM from https://github.com/zhengruifeng/spark-libFM <br><br>
Run the following Field-aware Factorization Machine test: <br><br>
https://github.com/zhengruifeng/spark-libFM/blob/master/src/main/scala/TestFM.scala <br><br>
Describe the dataset. Describe the two experiments: fm1 and fm2 and discuss your results.**

**Dataset:**<br>
The dataset comprises approximately 2.4M lines of the format: <br>
*label feature0:value0 feature1:value1 ... featureN:valueN*

The lines are in sparse format, the labels take values -1 or 1, and the feature values are either binary or take a fractional value between 0 and 1. The total uncompressed data size is about 2.2G.

**Differences in the two factorizations:**
* fm1 trains an FFM using stochastic gradient descent for 100 iterations. This is an iterative method that processes one data point at a time and adjusts the model based on the gradient of the loss function. fm1 uses all of the data in each iteration (it can be configured to use lesS).
* fm2 trains an FFM using limited-memory Broyden–Fletcher–Goldfarb–Shanno for 20 iterations. This algorithm approximates Newton's method (using the gradient and an approximate Hessian) to find a more direct path to convergence than gradient descent.

Both experiments use equivalent parameters otherwise, including for interactions (use global bias term, use one-way interactions, use 4 factors for pairwise interactions) and regularization (all regularization parameters set to 0). There is no output from FFMTest, but we can compare runtimes: with 10 m3.xlarge instances, fm1 required approximately 2 minutes and fm2 13 minutes (15 minutes total).

The original scala code provides no output, so I wrote a function and created an additional RDD calculate log loss. Both fm1 and fm2 had identical log loss of 0.8520897668623415.

Below are the commands and code used to set up the cluster and compile and run the code.

In [ ]:
# set up a cluster of 10 m3.xlarge instances
./spark/ec2/spark-ec2 --key-pair=jamesr261 --identity-file=jamesr261.pem -s 12 -t m3.xlarge \
    --region=us-west-1 --zone=us-west-1a launch jr

This is the scala file for running the factorizations.

In [ ]:
import com.github.fommil.netlib.BLAS._
import com.github.fommil.netlib.BLAS.{getInstance => blas}

import scala.util.Random

import org.apache.spark.mllib.linalg._
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.mllib.regression._
import org.apache.spark.mllib.util.MLUtils

object TestFM extends App {

  override def main(args: Array[String]): Unit = {

    val sc = new SparkContext(new SparkConf().setAppName("TESTFM"))

    val training = MLUtils.loadLibSVMFile(sc, "s3n://<access_key>:<secret_key>@ucb-mids-mls-jamesroute-hw5/url_combined", false, -1, 20).cache()

    //    val task = args(1).toInt
    //    val numIterations = args(2).toInt
    //    val stepSize = args(3).toDouble
    //    val miniBatchFraction = args(4).toDouble

    val dataSize = training.count()

    // run with SGD, report log loss
    val fm1 = FMWithSGD.train(training, task = 1, numIterations = 100, stepSize = 0.15, miniBatchFraction = 1.0, dim = (true, true, 4), regParam = (0, 0, 0), initStd = 0.1)
    val logLoss_fm1 = training.map { point =>
      fm1.predict(point.features)
    }.sum() / dataSize
    println("training log loss for fm1 = " + logLoss_fm1)

    // run with LBFGS, report log loss 
    val fm2 = FMWithLBFGS.train(training, task = 1, numIterations = 20, numCorrections = 5, dim = (true, true, 4), regParam = (0, 0, 0), initStd = 0.1)
    val logLoss_fm2 = training.map { point =>
      fm2.predict(point.features)
    }.sum() / dataSize
    println("training log loss for fm2 = " + logLoss_fm2)

  }
}

This is the SBT file used to build the jar.

In [ ]:
name := "TESTFM"

version := "1.0"

scalaVersion := "2.10.4"

libraryDependencies ++= Seq(
  "org.apache.spark" % "spark-core_2.10" % "1.5.1",
  "org.apache.spark" % "spark-mllib_2.10" % "1.5.1"
)

This is the command to submit the job to Spark, run from the root of the project directory.

In [ ]:
# driver mem set to 6G to avoid java heap memory issues
~/spark/bin/spark-submit --driver-memory 6G \
    --master spark://ec2-52-53-250-84.us-west-1.compute.amazonaws.com:7077 \
    target/scala-2.10/testfm_2.10-1.0.jar 

###HW14.4 Replicate Criteo Challenge winning solution


This is code used to preprocess the raw dataset into one with 1000 hashed features, output in LibSVM format. If we get GBDT and feature hashing to work, this step will be unnecessary.

In [3]:
%%writefile preprocess.py
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.util import MLUtils
from pyspark import SparkContext, SparkConf

from math import log, exp
from collections import defaultdict
import hashlib
import sys

# Calculate a feature dictionary for an observation's features based on hashing
def hashFunction(numBuckets, rawFeats, printMapping=False):
    mapping = {}
    for ind, category in rawFeats:
        featureString = category + str(ind)
        mapping[featureString] = int(int(hashlib.md5(featureString).hexdigest(), 16) % numBuckets)
    if(printMapping): print mapping
    sparseFeatures = defaultdict(float)
    for bucket in mapping.values():
        sparseFeatures[bucket] += 1.0
    return dict(sparseFeatures)

# Converts a comma separated string into a list of (featureID, value) tuples
def parsePoint(point):
    feature_list = point.split(',')[1:]
    return [(idx, feature) for idx, feature in enumerate(feature_list)]

# Create a LabeledPoint for this observation using hashing
def parseHashPoint(point, numBuckets):
    # parse the points
    point_list = parsePoint(point)
    #get the label of the point
    label = point.split(',')[0]
    
    features = hashFunction(numBuckets, point_list)
    return LabeledPoint(label, SparseVector(numBuckets, features))


if __name__ == "__main__":
    sc = SparkContext(appName="preprocess")
    input_file=sys.argv[1]
    output=sys.argv[2]
    numBucketsCTR=int(sys.argv[3])

    # read in the files and parse tab to comma
    rawData = sc.textFile(input_file, 20).map(lambda x: x.replace('\t', ','))
    
    # cache the data
    rawData.cache()
    
    # hash the data
    hashData = rawData.map(lambda x: parseHashPoint(x, numBucketsCTR))
    MLUtils.saveAsLibSVMFile(hashData, output)

Overwriting preprocess.py


This is the scala file for running FFM on the training dataset and calculating log loss and AUC on the training set. It is compiled and run the same way as the code in 14.3.

In [ ]:
import com.github.fommil.netlib.BLAS._
import com.github.fommil.netlib.BLAS.{getInstance => blas}

import scala.util.Random

import org.apache.spark.rdd.RDD
import org.apache.spark.mllib.linalg._
import org.apache.spark.mllib.evaluation.RegressionMetrics
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.mllib.regression._
import org.apache.spark.mllib.util.MLUtils
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics


object TestFM extends App {

  /**
    * print log loss and AUC for a model and dataset
    * model: trained FMM model
    * data: RDD containing training/validation/test data
    * desc: string used as title of dataset
    */
  def getMetrics(model: FMModel,
                 data: RDD[LabeledPoint],
                 desc: String): Unit = {

    val dataSize = data.count()
    val preds = data.map { point =>
      val prediction = model.predict(point.features)
      (prediction, point.label)
    }

    val logLoss = preds.map { pred_label =>
      computeLogLoss(pred_label._1, pred_label._2)
      }.sum() / dataSize

    val metrics = new BinaryClassificationMetrics(preds)
    val auROC = metrics.areaUnderROC

    println(desc + " log loss = " + logLoss)
    println("Area under ROC = " + auROC)
  }

  /**
    * function to compute log loss
    * p: model probability for given data point
    * y: label for data point
    */
  def computeLogLoss(p: Double, y: Double): Double = {
    val epsilon = 10E-12
    var x = math.max(epsilon, p)
    x = math.min(1 - epsilon, p)
    val logLoss = y*math.log(x) + (1-y)*math.log(1-x)
    return -logLoss
  }

  override def main(args: Array[String]): Unit = {

    val sc = new SparkContext(new SparkConf().setAppName("TESTFM"))

    // process args. args 0-3 specify data locations. rest are hyperparams
    val train_in = args(0)
    val val_in = args(1)
    val test_in = args(2)
    val step_size = args(3).toDouble
    val reg = args(4).toDouble
    val num_iter = args(5).toInt

    // read in data
    val training = MLUtils.loadLibSVMFile(sc, train_in).cache()
    val validate = MLUtils.loadLibSVMFile(sc, val_in).cache()
    val test = MLUtils.loadLibSVMFile(sc, test_in).cache()

    // train model
    val fm1 = FMWithSGD.train(training, task = 1, numIterations = num_iter, stepSize = step_size, miniBatchFraction = 1.0, dim = (true, true, 4), regParam = (reg, reg, reg), initStd = 0.1)

    // get metrics against training, validation, and test sets
    getMetrics(fm1, training, "training")
    getMetrics(fm1, validate, "validation")
    getMetrics(fm1, test, "test")
  }
}

Sample command to run FFM on the cluster, from the project's root directory. The arguments are as follows:
1. training set location
2. validation set location
3. test set location
4. step size
5. regulation param
6. number of iterations

In [ ]:
../spark/bin/spark-submit --master spark://ec2-52-53-221-66.us-west-1.compute.amazonaws.com:7077 \
    target/scala-2.10/testfm_2.10-1.0.jar \
    s3n://<access_key>:<secret_key>@ucb-mids-mls-jamesroute-hw5/criteo-train-1000 \
    s3n://<access_key>:<secret_key>@ucb-mids-mls-jamesroute-hw5/criteo-valid-1000 \
    s3n://<access_key>:<secret_key>@ucb-mids-mls-jamesroute-hw5/criteo-test-1000 1.5 1E-6 100

###Results
We run a handful of tests on the data, with a total runtime of about 60 minutes on 12 m3.xlarge instances. For now, the datasets were generated by hashing the raw data into 1000 buckets. We chose the following hyperparameter values:
* **step size:** 0.15, 0.5, 1.25 or 1.5
* **regulation parameter:** 0, 1E-6, 1E-3
* **number of iterations:** 100

####Table 1: Training Data LogLoss and AUC Value

|Log Loss|AUC Value|Step Size|Reg Param|Number Iterations|
|-|-|-|-|-|
|0.5812651631286812|0.5475276055210444|0.15|0|100|
|0.5583212569433286|0.6148138022912754|0.5|0|100|
|**0.5318299784251057**|0.678486224625755|1.5|0|100|
|0.581282196844917|0.5462594537466401|0.15|1e-6|100|
|0.5562192738289558|0.6180344981091518|0.5|1e-6|100|
|0.5339110403087934|0.6771116793841314|1.25|1e-6|1.5|
|0.5788832000989815|0.5388756286238648|0.15|1e-3|100|
|0.5584669577340001|0.6145967945963438|0.5|1e-3|100|
|0.5324526740624218|**0.6789627684773109**|1.25|1e-3|100|

####Table 2: Validation Data LogLoss and AUC Value

|Log Loss|AUC Value|Step Size|Reg Param|Number Iterations|
|-|-|-|-|-|
|0.5814953783962474|0.5472909130297269|0.15|0|100|
|0.5583624771970057|0.6150076498035781|0.5|0|100|
|**0.531713162443586** |**0.6789444465955337**|1.5|0|100|
|0.581460859665691|0.5460948369447726|0.15|1e-6|100|
|0.5563839218794641|0.6180450252001811|0.5|1e-6|100|
|0.534213952245211|0.676735543217068|1.25|1e-6|1.5|
|0.5790336852807052|0.5387405003639277|0.15|1e-3|100|
|0.5586476671536188|0.6146443231524853|0.5|1e-3|100|
|0.532672536417834|0.6787517531915529|1.25|1e-3|100|

####Table 3: Test Data LogLoss and AUC Value

|Log Loss|AUC Value|Step Size|Reg Param|Number Iterations|
|-|-|-|-|-|
|0.5815648444739806|0.5476740008575337|0.15|0|100|
|0.5584779781283143|0.6153536713548916|0.5|0|100|
|**0.5318905726796802**|0.678818844239985|1.5|0|100|
|0.5815725093404048|0.5463220966180165|0.15|1e-6|100|
|0.5565032761659852|0.618282460615438|0.5|1e-6|100|
|0.5339955470930027|0.677391148251628|1.25|1e-6|1.5|
|0.5791791356349545|0.5389986208305242|0.15|1e-3|100|
|0.5586332832864396|0.6149699808211845|0.5|1e-3|100|
|0.5324809477467553|**0.6793974696153768**|1.25|1e-3|100|

From the tables above, we can see that the step size of 1.5 and regulation parameter of 0 yielded the lowest log loss. A step size of 1.25 and regulation parameter of 1E-3 yielded the highest AUC.

We get NaN for step size of 1.5 and non-zero regulation parameter, suggesting that the model cannot tolerate a step size this high in some circumstances. We therefore substituted a step size of 1.25 for these cases.

And now for our final action in 261, we shut down the cluster.

In [ ]:
# time to close it down
./spark/ec2/spark-ec2 --region=us-west-1 destroy jr